## Inference using unsloth ai

In [1]:
###########################
import os
def gpu_info():
    dat=[]
    import subprocess
    out = subprocess.run(["nvidia-smi"], capture_output=True, text=True).stdout
    out2 = out.split("\n")[8:]
    splitter = out2[3]
    out3 = "\n".join(out2).split(splitter)[:-1]

    # Check GPU usage
    for idx,x in enumerate(out3):
        gpu = ((x.split("|"))[1].replace("Off", ""))
        t = ((x.split("|"))[6].replace("Off", "").replace("MiB", "").split("/"))
        ans = (int(t[0]) / int(t[1])) * 100
        dat.append({
                "gpu_id": idx,
                "gpu": ((gpu[:-5])[6:]).strip(),
                "usage": f"{ans:.2f}"
            })
    return dat
gpu_info_ans = gpu_info()
flag=0
for gpu in gpu_info_ans:
    if float(gpu['usage'])<10: 
        empty_gpu = gpu['gpu_id']
        flag=1
        break
if flag==0:
    print("No empty GPU found, please check your GPU usage.")
    exit(1)
else:
    print(f"Using GPU {empty_gpu} for processing.")

os.environ["CUDA_VISIBLE_DEVICES"] = str(empty_gpu)
############################

Using GPU 2 for processing.


In [ ]:

from unsloth import FastLanguageModel
model_name = "/home/mshahidul/project1/model/unsloth/Qwen2.5-14B-Instruct_alpaca"
# model_name = "Qwen2.5-14B-Instruct"
max_seq_length,dtype,load_in_4bit,model_name = 4092,None ,True, 
model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = model_name, # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = None,
        load_in_4bit = True,
    )
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
                    ### Instruction:
                    {}

                    ### Input:
                    {}

                    ### Response:
                    {}"""
prompt='Translate the following English text into Spanish naturally and accurately:'
ques = "The cat is on the roof."
inputs = tokenizer(
    [
        alpaca_prompt.format(
            f"{prompt}", # instruction
            ques, # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 256, temperature = 0.3, top_p = 0.9, top_k = 32,do_sample=True)
ans=tokenizer.batch_decode(outputs)
start_marker = '### Response:\n'
end_marker = '<|im_end|'
start_index = ans[0].find(start_marker) + len(start_marker)
end_index = ans[0].find(end_marker)
response = ans[0][start_index:end_index].strip()
print(f"Input: {ques}")
print(f"Output: {response}")

## vllm

In [ ]:
import sys
import os
sys.path.append(os.path.abspath('/home/mshahidul/'))
from gpu_selection import _gpu_selection_
_gpu_selection_()
from vllm import LLM, SamplingParams
import torch
llm = LLM(model="unsloth/Qwen2.5-14B-Instruct",dtype=torch.float16,quantization="bitsandbytes", load_format="bitsandbytes")


In [ ]:
prompt_template= f"Explain the large language models."
sampling_params = SamplingParams(temperature=1.0, top_p=0.95,max_tokens=256)
outputs = llm.generate(prompt_template, sampling_params)
for output in outputs:
    prompt = output.prompt
    response = output.outputs[0].text
    print(f"response: {response}")

CUDA_VISIBLE_DEVICES=6 python -m vllm.entrypoints.openai.api_server \
    --port 8000 \
    --model unsloth/Qwen2.5-14B-Instruct \
    --dtype float16 \
    --quantization bitsandbytes \
    --load-format bitsandbytes

## [VLLM OpenAI server](https://docs.vllm.ai/en/v0.8.3/serving/openai_compatible_server.html)

In [ ]:
from openai import OpenAI
client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="token-abc123",
)

completion = client.chat.completions.create(
  model="unsloth/Qwen2.5-14B-Instruct",
  messages=[
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)

wget https://huggingface.co/TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF/resolve/main/tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf
CUDA_VISIBLE_DEVICES=6 vllm serve ./tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf \
   --tokenizer TinyLlama/TinyLlama-1.1B-Chat-v1.0

In [ ]:
import sys
import os
sys.path.append(os.path.abspath('/home/mshahidul/'))
from gpu_selection import _gpu_selection_
_gpu_selection_()
from vllm import LLM, SamplingParams

# In this script, we demonstrate how to pass input to the chat method:
conversation = [
   {
      "role": "system",
      "content": "You are a helpful assistant"
   },
   {
      "role": "user",
      "content": "Hello"
   },
   {
      "role": "assistant",
      "content": "Hello! How can I assist you today?"
   },
   {
      "role": "user",
      "content": "Write an essay about the importance of higher education.",
   },
]

# Create a sampling params object.
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)

# Create an LLM.
llm = LLM(model="/home/mshahidul/tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf",
         tokenizer="TinyLlama/TinyLlama-1.1B-Chat-v1.0")
# Generate texts from the prompts. The output is a list of RequestOutput objects
# that contain the prompt, generated text, and other information.
outputs = llm.chat(conversation, sampling_params)

# Print the outputs.
for output in outputs:
   prompt = output.prompt
   generated_text = output.outputs[0].text
   print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

python /home/mshahidul/webiner/inference.py --finetune True --ques "The cut through the muscle is then closed horizontally to keep the pylorus open and allow the stomach to empty." 

In [2]:
import json

with open("/home/mshahidul/project1/all_tran_data/dataset/medline_data_for_finetune.json", "r") as f:
    medline_data = json.load(f)

medline_data[8]

{'english': 'The cut through the muscle is then closed horizontally to keep the pylorus open and allow the stomach to empty.',
 'spanish': 'Luego, se cierra horizontalmente la incisión a través del músculo para mantener el píloro abierto y permitir que el estómago se vacíe.'}